In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import random
import sys


In [2]:
# Download Shakespeare text
path = keras.utils.get_file(
    'shakespeare.txt',
    'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'
)

# Read the text
text = open(path, 'r').read()
print(f'Text length: {len(text)} characters')
print(text[:500])  # Preview first 500 characters


1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Text length: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [3]:
# Get unique characters and create mappings
chars = sorted(list(set(text)))
char2idx = {c:i for i, c in enumerate(chars)}
idx2char = {i:c for i, c in enumerate(chars)}

vocab_size = len(chars)
print(f'Unique characters: {vocab_size}')

# Convert text to integer sequence
text_as_int = np.array([char2idx[c] for c in text])

# Sequence length
seq_length = 100
examples_per_epoch = len(text)//seq_length

# Create input-target sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]  # all except last char
    target_text = chunk[1:]  # all except first char
    return input_text, target_text

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


Unique characters: 65


In [7]:
# ----------------------------
# Cell 4 — Build the LSTM Model (Fixed)
# ----------------------------

embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units):
    model = keras.Sequential([
        # Embedding layer
        layers.Embedding(vocab_size, embedding_dim),

        # LSTM (not stateful)
        layers.LSTM(
            rnn_units,
            return_sequences=True,
            recurrent_initializer='glorot_uniform'
        ),

        # Output Dense
        layers.Dense(vocab_size)
    ])
    return model

# Build the model
model = build_model(vocab_size, embedding_dim, rnn_units)
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
def loss(labels, logits):
    return keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)


In [9]:
EPOCHS = 5

history = model.fit(dataset, epochs=EPOCHS)


Epoch 1/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 17s 66ms/step - loss: 2.8751
Epoch 2/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 68ms/step - loss: 1.8477
Epoch 3/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - loss: 1.5848
Epoch 4/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 72ms/step - loss: 1.4488
Epoch 5/5
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 74ms/step - loss: 1.3701


In [10]:
def generate_text(model, start_string, num_generate=500, temperature=1.0):
    # Convert start string to numbers
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    # Batch size 1 for generation
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Use temperature to control randomness
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Add predicted char to generated text
        text_generated.append(idx2char[predicted_id])

        # Use predicted char as next input
        input_eval = tf.expand_dims([predicted_id], 0)

    return start_string + ''.join(text_generated)


In [13]:
# ----------------------------
# Cell 8 — Generate Text (Fixed for non-stateful LSTM)
# ----------------------------

def generate_text(model, start_string, num_generate=500, temperature=1.0):
    # Convert start string to numbers
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        # Use temperature to control randomness
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Add predicted char to generated text
        text_generated.append(idx2char[predicted_id])

        # Use predicted char as next input
        input_eval = tf.expand_dims([predicted_id], 0)

    return start_string + ''.join(text_generated)

# Generate text
print(generate_text(model, start_string="ROMEO: ", num_generate=500, temperature=0.5))


ROMEO: I e poris she the me the inere wousthathanouthan thouthe the t want hangoure whanoous the thuthe be wanou athalle an me he gre tore fine here in in s thee the pe the t the tha allithe s sth the t f he abus te me the the the are t mere merd me malo se mathe t the ghat ice thatherere ongranoure t the s the whe hacome be be t t w t at ther te greth I hare nghe oure mer the be is t s we ano ange be be he t ar lite wen s t the an s uthin Thacow the ther the ise benen anoure thous, s southe ss th thel
